In [3]:
import ee
import datetime

# --- IMPORTANT: Ensure you have authenticated with 'ee.Authenticate()' before running. ---
# --- This script is for joining and downloading two MODIS collections. ---

# Initialize Earth Engine
try:
    ee.Initialize(project='ee-ilyasnursyamsi05')
    print("Earth Engine initialized successfully.")
except ee.EEException as e:
    print(f"Error initializing Earth Engine: {e}")
    print("Please ensure you have authenticated with 'ee.Authenticate()' and re-run.")
    exit()

# --- 1. Define ROI ---
ROI = ee.FeatureCollection('projects/ee-ilyasnursyamsi05/assets/CHC')

# --- 2. Load Base MODIS Image Collections with New Date Filters ---
print('Filtering MODIS data for October and November from 2015-2024...')

# Load the base collections
modisA4 = ee.ImageCollection('MODIS/061/MCD43A4').filterBounds(ROI)
modisA2 = ee.ImageCollection('MODIS/061/MCD43A2').filterBounds(ROI)

# Define the years and months for filtering
years = range(2015, 2025)
months = [10, 11]

# Build a list of date filters
date_filters = []
for year in years:
    for month in months:
        start_date = ee.Date.fromYMD(year, month, 1)
        end_date = start_date.advance(1, 'month')
        date_filters.append(ee.Filter.date(start_date, end_date))

# Combine all the date filters into a single filter using 'Or'
combined_date_filter = ee.Filter.Or(date_filters)

# Apply the combined filter to the base collections
modisA4_filtered = modisA4.filter(combined_date_filter)
modisA2_filtered = modisA2.filter(combined_date_filter)

# --- Debugging: Print initial collection sizes after filtering ---
print('Number of MCD43A4 images after filtering:', modisA4_filtered.size().getInfo())
print('Number of MCD43A2 images after filtering:', modisA2_filtered.size().getInfo())

# --- 3. Prepare A2 Collection: Select & Rename Bands ---
def rename_a2_bands(image):
  """Renames A2 quality bands for clarity during merging."""
  return image.select(
    ['BRDF_Albedo_LandWaterType', 'Snow_BRDF_Albedo'],
    ['QA_LandWaterType', 'QA_Snow']
  )

modisA2_renamed = modisA2_filtered.map(rename_a2_bands)

# --- 4. Define All Bands to Export ---
bands_a4 = [
    'Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band3',
    'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band5', 'Nadir_Reflectance_Band6',
    'Nadir_Reflectance_Band7'
]
all_bands_to_export = bands_a4 + ['QA_LandWaterType', 'QA_Snow']

# --- 5. Join MCD43A4 and MCD43A2 Image Collections by Date ---
filter_time_eq = ee.Filter.equals(
  leftField='system:time_start',
  rightField='system:time_start'
)

inner_join = ee.Join.inner('primary', 'secondary')

joined_images = inner_join.apply(modisA4_filtered, modisA2_renamed, filter_time_eq)

def combine_bands_from_joined(feature):
  """Combines bands from the joined A4 and A2 images."""
  primary = ee.Image(feature.get('primary'))
  secondary = ee.Image(feature.get('secondary'))
  return primary.addBands(secondary)

combined_collection = joined_images.map(combine_bands_from_joined)

# --- Debugging: Print combined collection size ---
combined_size = combined_collection.size().getInfo()
print('Number of combined images after joining:', combined_size)

# --- 6. Export the Entire Combined ImageCollection to Google Drive ---
if combined_size == 0:
    print("Combined collection is empty, skipping export tasks to Google Drive.")
else:
    # Get the list of images to iterate and export
    image_list = combined_collection.toList(combined_size)

    # Iterate over the images in the list and create an export task for each
    for i in range(combined_size):
        image = ee.Image(image_list.get(i)) # Get the image by index
        date_str = ee.Date(image.get('system:time_start')).format('YYYY_MM_dd').getInfo()
        file_name_prefix = 'MODIS_Combined_' + date_str

        # Get projection information from the image itself (still useful for native scale)
        projection = image.select(all_bands_to_export[0]).projection()
        scale = projection.nominalScale().getInfo() # Get the native resolution (500m)

        task = ee.batch.Export.image.toDrive(
            image=image.select(all_bands_to_export).float(),
            description=file_name_prefix + '_export',
            folder='GEE_MODIS_Exports_CHC',
            fileNamePrefix= file_name_prefix,
            scale=scale,
            region=ROI.geometry(),
            fileFormat='GeoTIFF',
            crs='EPSG:4326',
            maxPixels=1e13
        )
        task.start()
        print(f"Export task initiated for: {file_name_prefix}. Check your GEE Tasks tab.")

Earth Engine initialized successfully.
Filtering MODIS data for October and November from 2015-2024...
Number of MCD43A4 images after filtering: 610
Number of MCD43A2 images after filtering: 610
Number of combined images after joining: 610
Export task initiated for: MODIS_Combined_2015_10_01. Check your GEE Tasks tab.
Export task initiated for: MODIS_Combined_2015_10_02. Check your GEE Tasks tab.
Export task initiated for: MODIS_Combined_2015_10_03. Check your GEE Tasks tab.
Export task initiated for: MODIS_Combined_2015_10_04. Check your GEE Tasks tab.
Export task initiated for: MODIS_Combined_2015_10_05. Check your GEE Tasks tab.
Export task initiated for: MODIS_Combined_2015_10_06. Check your GEE Tasks tab.
Export task initiated for: MODIS_Combined_2015_10_07. Check your GEE Tasks tab.
Export task initiated for: MODIS_Combined_2015_10_08. Check your GEE Tasks tab.
Export task initiated for: MODIS_Combined_2015_10_09. Check your GEE Tasks tab.
Export task initiated for: MODIS_Combine

ConnectionError: HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Max retries exceeded with url: /v1/projects/ee-ilyasnursyamsi05/value:compute?prettyPrint=false&alt=json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000025288C7DA10>: Failed to resolve 'earthengine.googleapis.com' ([Errno 11001] getaddrinfo failed)"))